# 4.9 Data Visualizations (Part 1)

## This script contains the following points:
- Observation of a new customers dataframe 
- Data wrangling of customers dataframe (renaming columns)
- Data consistency check (mixed-type data, missing values, duplicates)
- Data combining (merging ords_prods_grouped dataframe with customers dataframe)
- Exporting new merged dataframe as a pickle file

In [1]:
# Import libraries
import pandas as pd
import numpy as np
import os

In [2]:
# Path to the original data
path= r"/Users/mentaykoshzhanova/Desktop/Instacart Basket Analysis"
path

'/Users/mentaykoshzhanova/Desktop/Instacart Basket Analysis'

In [3]:
# Import dataframe
customers= pd.read_csv(os.path.join(path, '02 Data', 'Original Data', 'customers.csv'), index_col=False)
ords_prods_grouped= pd.read_pickle(os.path.join(path, '02 Data', 'Prepared Data', 'orders_products_grouped.pkl'))

### 3. Observing customers df by using shape, info, describe, head functions:

In [4]:
customers.shape

(206209, 10)

In [5]:
customers.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 206209 entries, 0 to 206208
Data columns (total 10 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   user_id       206209 non-null  int64 
 1   First Name    194950 non-null  object
 2   Surnam        206209 non-null  object
 3   Gender        206209 non-null  object
 4   STATE         206209 non-null  object
 5   Age           206209 non-null  int64 
 6   date_joined   206209 non-null  object
 7   n_dependants  206209 non-null  int64 
 8   fam_status    206209 non-null  object
 9   income        206209 non-null  int64 
dtypes: int64(4), object(6)
memory usage: 15.7+ MB


In [6]:
# After running info function investigating further first name column with value counts, info shows there are missing values
customers['First Name'].value_counts(dropna = False)

NaN        11259
Marilyn     2213
Barbara     2154
Todd        2113
Jeremy      2104
           ...  
Merry        197
Eugene       197
Garry        191
Ned          186
David        186
Name: First Name, Length: 208, dtype: int64

In [7]:
customers.describe()

,user_id,Age,n_dependants,income
count,206209.000000,206209.000000,206209.000000,206209.000000
mean,103105.000000,49.501646,1.499823,94632.852548
std,59527.555167,18.480962,1.118433,42473.786988
min,1.000000,18.000000,0.000000,25903.000000
25%,51553.000000,33.000000,0.000000,59874.000000
50%,103105.000000,49.000000,1.000000,93547.000000
75%,154657.000000,66.000000,3.000000,124244.000000
max,206209.000000,81.000000,3.000000,593901.000000


In [8]:
customers.head(10)

,user_id,First Name,Surnam,Gender,STATE,Age,date_joined,n_dependants,fam_status,income
0,26711,Deborah,Esquivel,Female,Missouri,48,1/1/2017,3,married,165665
1,33890,Patricia,Hart,Female,New Mexico,36,1/1/2017,0,single,59285
2,65803,Kenneth,Farley,Male,Idaho,35,1/1/2017,2,married,99568
3,125935,Michelle,Hicks,Female,Iowa,40,1/1/2017,0,single,42049
4,130797,Ann,Gilmore,Female,Maryland,26,1/1/2017,1,married,40374
5,133128,Cynthia,Noble,Female,Kentucky,43,1/1/2017,2,married,49643
6,152052,Chris,Walton,Male,Montana,20,1/1/2017,0,single,61746
7,168851,Joseph,Hickman,Male,South Carolina,30,1/1/2017,0,single,63712
8,69965,Jeremy,Vang,Male,Texas,47,1/1/2017,1,married,162432
9,82820,Shawn,Chung,Male,Virginia,26,1/1/2017,2,married,32072


### 4. Data wrangling so it follows consistent logic 

## Observations:

- There are 11259 empty cells in first name column, which would be 5% of customers df first name column. For now I would leave them as it is, cells can be filled when get info from stakeholders.
- Age, number of dependants and income columns' descriptive statistics look good, nothing seems to be off or an outlier.
- User_id shows as integer but I am leaving as it is because of merging customers df to ords_prods df
- Column names are in different styles which would be changed to follow consistency.


### Renaming columns

In [9]:
# Renaming column Surnam to Last_name
customers.rename(columns = {'Surnam' : 'Last_name'}, inplace = True)

In [10]:
# Renaming column First Name to First_name
customers.rename(columns = {'First Name' : 'First_name'}, inplace = True)

In [11]:
# Renaming column STATE to State
customers.rename(columns = {'STATE' : 'State'}, inplace = True)

In [12]:
# Renaming date_joined to Date_joined
customers.rename(columns = {'date_joined' : 'Date_joined'}, inplace = True)

In [13]:
# Renaming n_dependants to Number_of_dependants
customers.rename(columns = {'n_dependants' : 'Number_of_dependants'}, inplace = True)

In [14]:
# Renaming fam_status to Marriage_status
customers.rename(columns = {'fam_status' : 'Marriage_status'}, inplace = True)

In [15]:
# Renaming income to Income
customers.rename(columns = {'income' : 'Income'}, inplace = True)

In [16]:
# Renaming user id (for merge)
customers.rename(columns={'User_id': 'user_id'}, inplace = True)

In [17]:
customers.head()

,user_id,First_name,Last_name,Gender,State,Age,Date_joined,Number_of_dependants,Marriage_status,Income
0,26711,Deborah,Esquivel,Female,Missouri,48,1/1/2017,3,married,165665
1,33890,Patricia,Hart,Female,New Mexico,36,1/1/2017,0,single,59285
2,65803,Kenneth,Farley,Male,Idaho,35,1/1/2017,2,married,99568
3,125935,Michelle,Hicks,Female,Iowa,40,1/1/2017,0,single,42049
4,130797,Ann,Gilmore,Female,Maryland,26,1/1/2017,1,married,40374


## 5. Data Consistency check

#### Finding mixed-type data and addressing it

In [18]:
for col in customers.columns.tolist():
  weird = (customers[[col]].applymap(type) != customers[[col]].iloc[0].apply(type)).any(axis = 1)
  if len (customers[weird]) > 0:
    print (col)


First_name


In [19]:
# Column first name has mixed data type because there are null values in the column. 
# Datatype can be changed to string and check after datatype change with dtype function
customers['First_name'] = customers['First_name'].astype('str')
customers['First_name'].dtype

dtype('O')

#### Finding missing values

In [20]:
# Finding missing data in customers df
customers.isnull().sum()

user_id                 0
First_name              0
Last_name               0
Gender                  0
State                   0
Age                     0
Date_joined             0
Number_of_dependants    0
Marriage_status         0
Income                  0
dtype: int64

In [21]:
# No missing data in df

#### Finding duplicates 

In [22]:
# Finding duplicates
customers_dups = customers[customers.duplicated()]

In [23]:
customers_dups

,user_id,First_name,Last_name,Gender,State,Age,Date_joined,Number_of_dependants,Marriage_status,Income


In [24]:
# No duplicates in df

## 6. Data combining

In [25]:
# Getting info of ords_prods_grouped df and customers df to find the common column to combine them
ords_prods_grouped.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 32434212 entries, 0 to 32434211
Data columns (total 28 columns):
 #   Column                  Dtype   
---  ------                  -----   
 0   order_id                int64   
 1   product_id              int64   
 2   add_to_cart_order       int64   
 3   reordered               int64   
 4   Unnamed: 0_x            int64   
 5   user_id                 int64   
 6   order_number            int64   
 7   order_day_of_week       int64   
 8   order_time              int64   
 9   days_since_prior_order  float64 
 10  _merge                  category
 11  Unnamed: 0_y            int64   
 12  product_name            object  
 13  aisle_id                int64   
 14  department_id           int64   
 15  prices                  float64 
 16  exists                  category
 17  price_range_loc         object  
 18  price_ranged_loc        object  
 19  Busiest day             object  
 20  Busiest days            object  
 21  Busies

In [26]:
customers.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 206209 entries, 0 to 206208
Data columns (total 10 columns):
 #   Column                Non-Null Count   Dtype 
---  ------                --------------   ----- 
 0   user_id               206209 non-null  int64 
 1   First_name            206209 non-null  object
 2   Last_name             206209 non-null  object
 3   Gender                206209 non-null  object
 4   State                 206209 non-null  object
 5   Age                   206209 non-null  int64 
 6   Date_joined           206209 non-null  object
 7   Number_of_dependants  206209 non-null  int64 
 8   Marriage_status       206209 non-null  object
 9   Income                206209 non-null  int64 
dtypes: int64(4), object(6)
memory usage: 15.7+ MB


In [27]:
# Merging ords_prods_grouped df with customers df on user id as a common column
df_merged = ords_prods_grouped.merge(customers, on = 'user_id')

In [28]:
# Running head function for a new merged df 
df_merged.head(10)

,order_id,product_id,add_to_cart_order,reordered,Unnamed: 0_x,user_id,order_number,order_day_of_week,order_time,days_since_prior_order,...,order_frequency,First_name,Last_name,Gender,State,Age,Date_joined,Number_of_dependants,Marriage_status,Income
0,2,33120,1,1,3355525,202279,3,5,9,8.0,...,Non-frequent customer,Paul,Coleman,Male,Idaho,57,2/6/2020,3,married,98119
1,104690,33120,2,1,3355530,202279,8,6,10,30.0,...,Non-frequent customer,Paul,Coleman,Male,Idaho,57,2/6/2020,3,married,98119
2,132412,33120,1,1,3355528,202279,6,5,9,30.0,...,Non-frequent customer,Paul,Coleman,Male,Idaho,57,2/6/2020,3,married,98119
3,2808715,33120,1,1,3355527,202279,5,5,13,9.0,...,Non-frequent customer,Paul,Coleman,Male,Idaho,57,2/6/2020,3,married,98119
4,2894949,33120,5,0,3355523,202279,1,5,9,NaN,...,Non-frequent customer,Paul,Coleman,Male,Idaho,57,2/6/2020,3,married,98119
5,2,28985,2,1,3355525,202279,3,5,9,8.0,...,Non-frequent customer,Paul,Coleman,Male,Idaho,57,2/6/2020,3,married,98119
6,132412,28985,7,1,3355528,202279,6,5,9,30.0,...,Non-frequent customer,Paul,Coleman,Male,Idaho,57,2/6/2020,3,married,98119
7,1500071,28985,1,1,3355524,202279,2,4,13,20.0,...,Non-frequent customer,Paul,Coleman,Male,Idaho,57,2/6/2020,3,married,98119
8,2808715,28985,2,1,3355527,202279,5,5,13,9.0,...,Non-frequent customer,Paul,Coleman,Male,Idaho,57,2/6/2020,3,married,98119
9,2894949,28985,4,0,3355523,202279,1,5,9,NaN,...,Non-frequent customer,Paul,Coleman,Male,Idaho,57,2/6/2020,3,married,98119


## Exporting Data

In [29]:
# Exporting new merged df as a pickle file
df_merged.to_pickle(os.path.join(path, '02 Data', 'Prepared Data', 'orders_products_customers_merged.pkl'))